Pulling the QuestDB image and creating a docker container

In [ ]:
!docker run -p 9000:9000 -p 8812:8812 questdb/questdb

Create db

In [ ]:
import requests
import urllib.parse as par

q = 'create table weather'\
    '(temp int,'\
    'rain24H double,'\
    'thunder boolean,'\
    'timestamp timestamp)'\
    'timestamp(timestamp)'
r = requests.get("http://localhost:9000/exec?query=" + q)
print(r.status_code)

Adding data to QuestDB

In [ ]:
import requests
import random
from datetime import datetime
success = 0
fail = 0
random.seed()
for x in range(1000):
    temp = random.randint(-40, 55)
    rain24H = round(random.uniform(10.45, 235.15), 2)
    thunder = bool(random.getrandbits(1))
    query = "insert into weather values("\
        + str(temp) + ","\
        + str(rain24H) + "," \
        + str(thunder) +",systimestamp())"
    r = requests.get("http://localhost:9000/exec?query=" + query)
    if r.status_code == 200:
        success += 1
    else:
        fail += 1

print("Rows inserted: " + str(success))
if fail > 0:
    print("Rows Failed: " + str(fail))

Query data from QuestDB

In [ ]:
import requests
import io

r = requests.get("http://localhost:9000/exp?query=select * from weather")
rawData = r.text
print(rawData)

Output

"tempF","rain24H","thunder","timestamp"
-37,234.0,false,"2020-08-24T11:41:37.191530Z"
-11,165.52,false,"2020-08-24T11:41:37.195714Z"
-31,178.13,true,"2020-08-24T11:41:37.202453Z"
14,31.470000000000,true,"2020-08-24T11:41:37.208365Z"
33,83.31,true,"2020-08-24T11:41:37.214347Z"

Convert to df

In [ ]:
import pandas as pd

pData = pd.read_csv(io.StringIO(rawData), parse_dates=['timestamp'])
print(pData)

Output:

      tempF  rain24H  thunder                        timestamp
0       -37   234.00    False 2020-08-24 11:41:37.191530+00:00
1       -11   165.52    False 2020-08-24 11:41:37.195714+00:00
2       -31   178.13     True 2020-08-24 11:41:37.202453+00:00
3        14    31.47     True 2020-08-24 11:41:37.208365+00:00
4        33    83.31     True 2020-08-24 11:41:37.214347+00:00
...     ...      ...      ...                              ...
3995    -24   183.22    False 2020-08-24 11:58:32.164254+00:00
3996      1   151.96     True 2020-08-24 11:58:32.166610+00:00
3997    -40   213.86     True 2020-08-24 11:58:32.170839+00:00
3998    -33   101.91     True 2020-08-24 11:58:32.173131+00:00
3999    -38   182.98    False 2020-08-24 11:58:32.177191+00:00

[4000 rows x 4 columns]

The query string must be URL-encoded before it is sent.

In [ ]:
import urllib.parse

q = "select tempF,"\
    " rain24H,"\
    " timestamp"\
    " from weather"\

query = urllib.parse.quote(q)
r = requests.get("http://localhost:9000/exp?query=" + query)
queryData = r.content
rawData = pd.read_csv(io.StringIO(queryData.decode('utf-8')))
print(rawData)

Output:

      tempF  rain24H                    timestamp
0       -37   234.00  2020-08-24T11:41:37.191530Z
1       -11   165.52  2020-08-24T11:41:37.195714Z
2       -31   178.13  2020-08-24T11:41:37.202453Z
3        14    31.47  2020-08-24T11:41:37.208365Z
4        33    83.31  2020-08-24T11:41:37.214347Z
...     ...      ...                          ...
3995    -24   183.22  2020-08-24T11:58:32.164254Z
3996      1   151.96  2020-08-24T11:58:32.166610Z
3997    -40   213.86  2020-08-24T11:58:32.170839Z
3998    -33   101.91  2020-08-24T11:58:32.173131Z
3999    -38   182.98  2020-08-24T11:58:32.177191Z

[4000 rows x 3 columns]

Plotting data using matplotlib

In [ ]:
from matplotlib import pyplot as plt

plt.bar(rawData['timestamp'], rawData['rain24H'])

In [ ]:
from matplotlib import pyplot as plt

plt.bar(rawData['timestamp'], rawData['tempF'])